In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from urllib.parse import urlparse
from preprocess_datasets import all_dataset
import torch.nn.functional as F
import torch.optim as optim
import os
import json
import matplotlib.pyplot as plt
from datetime import datetime
import time


import random
tqdm.pandas()
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"✅ Seed fixed to {seed}")

Using the latest cached version of the dataset since kmack/Phishing_urls couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\rrpra\.cache\huggingface\datasets\kmack___phishing_urls\default\0.0.0\ef1558ba51b6b3e1a2c264ec8227be0a08736e10 (last modified on Sat Oct  4 18:12:33 2025).
Using the latest cached version of the dataset since kmack/Phishing_urls couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\rrpra\.cache\huggingface\datasets\kmack___phishing_urls\default\0.0.0\ef1558ba51b6b3e1a2c264ec8227be0a08736e10 (last modified on Sat Oct  4 18:12:33 2025).


✅ Seed fixed to 42


In [6]:
# ============================================================
# Character Encoding Setup
# ============================================================

# Allowed printable ASCII chars
ascii_chars = [chr(i) for i in range(32, 127)]

# Special control tokens
special_tokens = [
    '<PAD>', '<UNK>',
]

# Build vocab and mapping
vocab = special_tokens + ascii_chars
char2idx = {ch: i for i, ch in enumerate(vocab)}



def encode(text, max_len=60):
    indices = torch.full((max_len,), char2idx["<PAD>"], dtype=torch.long)
    text = text.lower()[:max_len]
    for i, c in enumerate(text):
        indices[i] = char2idx.get(c, char2idx["<UNK>"])

    return indices





encoded_data = {}
frac = 1
gen = all_dataset()

x=0

max_len=60
next(gen)
next(gen)
next(gen)
for name, splits in  gen:
    encoded_data[name] = {}
    print(f"\n🔠 Encoding structured URLs for {name}...")

    for split_name, df in zip(['train', 'valid', 'test'], splits):
        df = df.sample(frac=frac, random_state=42)
        df["encode"] = df["url"].progress_apply(encode)
        encoded_data[name][split_name] = df
        print(f"  ✅ {split_name}: Encoded {len(df)} URLs")

    x+=1
    if x > 0:
        next(gen)


print("\n✅ All datasets encoded with proper start/end markers and padding!")


🔠 Encoding structured URLs for Dataset 4 (kaggels/taruntiwarihp/phishing-site-urls)...


100%|██████████| 405756/405756 [01:01<00:00, 6644.24it/s]


  ✅ train: Encoded 405756 URLs


100%|██████████| 50719/50719 [00:07<00:00, 6529.87it/s]


  ✅ valid: Encoded 50719 URLs


100%|██████████| 50720/50720 [00:08<00:00, 6031.52it/s]

  ✅ test: Encoded 50720 URLs

✅ All datasets encoded with proper start/end markers and padding!


In [ ]:
# ============================================================
# Convert to TensorDataset and DataLoader
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 524

dataloader_dict = {}

def make_tensor_dataset(df):
    url_tensor = torch.stack(list(df["encode"]))
    labels_tensor = torch.tensor(df["label"].values, dtype=torch.long)
    return TensorDataset(url_tensor, labels_tensor)

for name, splits in encoded_data.items():
    dataloader_dict[name] = {}
    print(f"\n📦 Creating DataLoaders for {name}...")
    
    train_set = make_tensor_dataset(splits["train"])
    val_set = make_tensor_dataset(splits["valid"])
    test_set = make_tensor_dataset(splits["test"])
    
    dataloader_dict[name]["train_loader"] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    dataloader_dict[name]["val_loader"] = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    dataloader_dict[name]["test_loader"] = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    
    print(f"✅ DataLoaders ready for {name} (Train/Val/Test)")

print("\n🚀 All DataLoaders are ready in `dataloader_dict`!")

# Example Access:
# dataloader_dict["dataset1"]["train_loader"]



📦 Creating DataLoaders for Dataset 4 (kaggels/taruntiwarihp/phishing-site-urls)...
✅ DataLoaders ready for Dataset 4 (kaggels/taruntiwarihp/phishing-site-urls) (Train/Val/Test)

🚀 All DataLoaders are ready in `dataloader_dict`!


In [ ]:
# =====================================================
# 🔹  Embeding_layer
# =====================================================
class Embeding_layer(nn.Module):
    def __init__(self,
                 vocab_size=256,
                 d_model=128,
                 max_len=100,
                 n_out=128,):
        super().__init__()

        # 🔹 Byte embedding layer (0–255)
        self.embedding = nn.Embedding(vocab_size, d_model)

        # 🔹 Positional embeddings
        self.pos_embedding = nn.Embedding(max_len, d_model)

        # 🔹 Final normalization
        self.final_norm = nn.LayerNorm(d_model)
        self.projection = nn.Linear(in_features=d_model, out_features=n_out)

    def forward(self, x):
        """
        x: (batch, seq_len) — byte indices [0–255]
        """
        batch_size, seq_len = x.size()
        device = x.device

        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        x = self.embedding(x) + self.pos_embedding(positions)

        #x = self.encoder(x)
        #x = self.embedding(x)
        x = self.projection(self.final_norm(x))

        return x  # (B, L, d_model)

In [9]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.dropout = nn.Dropout(0.25)
        self.fc2 = nn.Linear(channels // reduction, channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, L)
        w = x.mean(dim=2)                          # Global Average Pooling -> (B, C)
        w = self.dropout(F.elu(self.fc1(w)))
        w = self.sigmoid(self.fc2(w))
        w = w.unsqueeze(2)                         # (B, C, 1)
        return x * w                               # scale features

In [10]:
# 🔹 Residual Depthwise-Separable Multi-Kernel Block
class ResidualConvBlockDW(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_sizes=[3, 5, 7], reduction=16):
        super().__init__()
        self.branches = nn.ModuleList()

        mid_ch = max(in_ch // 16, 8)  # reduce dimension before heavy convs

        for k in kernel_sizes:
            branch = nn.Sequential(
                # (B) Reduce channels first
                nn.Conv1d(in_ch, mid_ch, kernel_size=1, bias=False),
                #nn.BatchNorm1d(mid_ch),
                nn.ReLU(inplace=True),
                nn.Dropout1d(0.5),

                # (A) Depthwise conv
                nn.Conv1d(mid_ch, mid_ch, kernel_size=k, padding=k // 2, groups=mid_ch, bias=False),
                #nn.BatchNorm1d(mid_ch),
                nn.ReLU(inplace=True),
                nn.Dropout1d(0.5),

                # Pointwise to expand to out_ch
                nn.Conv1d(mid_ch, out_ch, kernel_size=1, bias=False),
                #nn.BatchNorm1d(out_ch),
                nn.ReLU(inplace=True)
            )
            self.branches.append(branch)

        # Combine all kernel branches
        self.merge_conv = nn.Conv1d(out_ch * len(kernel_sizes), out_ch, kernel_size=1, bias=False)
        #self.merge_bn = nn.BatchNorm1d(out_ch)
        
        self.se = SEBlock(out_ch, reduction)
        self.shortcut = nn.Conv1d(in_ch, out_ch, kernel_size=1) if in_ch != out_ch else nn.Identity()

    def forward(self, x):
        # Parallel multi-kernel branches
        out = [branch(x) for branch in self.branches]
        out = torch.cat(out, dim=1)

        out = self.merge_conv(out)
        #out = self.merge_bn(out)
        out = self.se(out)
        out += self.shortcut(x)
        return F.relu(out)

    


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class URLBinaryCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, maxlen=100):
        super(URLBinaryCNN, self).__init__()
        self.maxlen = maxlen

        self.embeding_layer = nn.ModuleDict({
            "embeding":  Embeding_layer(vocab_size=vocab_size, max_len=maxlen, d_model=256, n_out=embed_dim)
        })

        # 2️⃣ Shared Layer (Global)
        self.shared_layer = nn.ModuleDict({
            "conv": ResidualConvBlockDW(embed_dim, 64, kernel_sizes=[3]),
            #"max_pool": nn.MaxPool1d(kernel_size=2),
            "bilstm": nn.LSTM(input_size=64, hidden_size=64, batch_first=True, bidirectional=True),
            "layer_norm": nn.LayerNorm(64*2),
            "fc1": nn.Linear(64 *2* maxlen , 64),
            "relu1": nn.ReLU(),
            "dropout1": nn.Dropout(0.5),
            
        })

        # 3️⃣ Personalization Layer (Local)
        self.personal_layer = nn.ModuleDict({
            "fc2": nn.Linear(64, 64),
            "relu2": nn.ReLU(),
            "dropout2": nn.Dropout(0.5),
            "head": nn.Linear(64, 1)
        })

    def forward(self, x):
        # Transformer
        x = self.embeding_layer["embeding"](x)
        #x = x.permute(0, 2, 1)
        # Shared layers
        #x = self.shared_layer["conv"](x)
        #x = self.shared_layer["max_pool"](x)
        #x = x.permute(0, 2, 1)
        x, _ = self.shared_layer["bilstm"](x)
        #x = self.shared_layer["layer_norm"](x)
        #x = x[:, -1, :]
        x = x.flatten(1)
        x = self.shared_layer["dropout1"](self.shared_layer["relu1"](self.shared_layer["fc1"](x)))
        # Personalization head
        x = self.personal_layer["dropout2"](self.personal_layer["relu2"](self.personal_layer["fc2"](x)))
        x = self.personal_layer["head"](x)
        return torch.sigmoid(x)
    
    def extract_features(self, x):
        """Return deep features before final FC layers."""
        # Transformer
        x = self.embeding_layer["embeding"](x)
        x = x.permute(0, 2, 1)
        # Shared layers
        x = self.shared_layer["conv"](x)
        x = x.permute(0, 2, 1)
        x, _ = self.shared_layer["bilstm"](x)
        x = self.shared_layer["layer_norm"](x)
        #x = x[:, -1, :]
        x = x.flatten(1)
        x = self.shared_layer["dropout1"](self.shared_layer["relu1"](self.shared_layer["fc1"](x)))
        return x



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, r2_score


class Train:
    def __init__(self, 
                 model, 
                 criterion, 
                 global_optimizer=None,
                 personal_optimizer = None, 
                 scheduler_g=None,
                 scheduler_p=None,
                 train_loader=None, 
                 val_loader=None,   
                 run_name="my_experiment",
                 save = 0,
                 device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        
        """
        optimizer_groups: dict with keys like {"transformer": optimizer1, "cnn": optimizer2}
        schedulers: dict with keys matching optimizer_groups (optional)
        """
        self.model = model
        self.best_val_loss = float('inf')
        self.best_train_loss = float('inf')
        self.criterion = criterion
        
        self.global_params = []
        self.persnalization_params = []
        for name, param in model.embeding_layer.named_parameters():
            self.global_params.append(param)
        for name, param in model.shared_layer.named_parameters():
            self.global_params.append(param)
        for name, param in model.personal_layer.named_parameters():
            self.persnalization_params.append(param)



        self.global_optimizer = optim.NAdam(self.global_params, lr=1e-4) if global_optimizer is None  else global_optimizer
        self.personal_optimizer = optim.NAdam(self.persnalization_params, lr=1e-3) if personal_optimizer is None else personal_optimizer
        self.scheduler_g = optim.lr_scheduler.ReduceLROnPlateau(self.global_optimizer, mode='min', factor=0.5, patience=2) if scheduler_g is None  else scheduler_g
        self.scheduler_p = optim.lr_scheduler.ReduceLROnPlateau(self.personal_optimizer, mode='min', factor=0.5, patience=2) if scheduler_p is None  else scheduler_p
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.batch_train_losses = []
        self.batch_train_accs = []
        self.epoch_train_losses = []
        self.epoch_train_accs = []
        self.epoch_val_losses = []
        self.epoch_val_accs = []

        self.batch_time = []
        self.epoch_time = []
        self.save = save
        self._create_run_folder(folder_name=run_name)
        self.best_model_path = os.path.join(self.run_folder, "best_model.pt")

        if self.save > 1: 
            # --------------------------------------------------------------------
            #                SAVE MODEL CONFIGURATION INTO JSON FILE
            # --------------------------------------------------------------------

            def count_params(module):
                return sum(p.numel() for p in module.parameters())

            def count_trainable(module):
                return sum(p.numel() for p in module.parameters() if p.requires_grad)

            model_config = {
                "run_name": run_name,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "manual_notes": '',

                # -------------------------
                # MODEL STRUCTURE
                # -------------------------
                "model_pipeline": str(model),  # string version
                "model_layers": {
                    "embedding_layer_params": count_params(model.embeding_layer),
                    "shared_layer_params": count_params(model.shared_layer),
                    "personal_layer_params": count_params(model.personal_layer),
                },

                # -------------------------
                # TRAINABLE / NON-TRAINABLE
                # -------------------------
                "trainable_parameters": count_trainable(model),
                "total_parameters": sum(p.numel() for p in model.parameters()),

                "global_parameter_count": len(self.global_params),
                "personal_parameter_count": len(self.persnalization_params),

                # -------------------------
                # OPTIMIZER / SCHEDULER SETTINGS
                # -------------------------
                "global_optimizer": str(self.global_optimizer),
                "personal_optimizer": str(self.personal_optimizer),
                "scheduler_global": str(self.scheduler_g),
                "scheduler_personal": str(self.scheduler_p),

                # -------------------------
                # TRAINING SETTINGS
                # -------------------------
                "batch_size": train_loader.batch_size if train_loader is not None else None,
                "num_train_batches": len(train_loader) if train_loader is not None else None,
                "num_val_batches": len(val_loader) if val_loader is not None else None,
            }

            # Save JSON config
            with open(os.path.join(self.run_folder, "model_configuration.json"), "w") as f:
                json.dump(model_config, f, indent=4)

            print("📁 Model configuration saved at:", os.path.join(self.run_folder, "model_configuration.json"))

    def freeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = False

    def unfreeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = True


    def train(self, epochs_list=[4,3,3], early_stopping=True, frac=1.0, val_frac=1.0,start=0,  log=0):
        train_length = len(self.train_loader)
        for phase, epochs in enumerate(epochs_list):
            for epoch in range(epochs):
                et0 = time.perf_counter()
                if phase == 0:
                    for param in self.model.embeding_layer.parameters():
                        param.requires_grad = True
                    for param in self.model.shared_layer.parameters():
                        param.requires_grad = True
                    for param in self.model.personal_layer.parameters():
                        param.requires_grad = True
                    active_optims = [self.global_optimizer, self.personal_optimizer]
                    active_scheds = [self.scheduler_g, self.scheduler_p]
                    phase_name = "global+personal"
                elif phase == 1:
                    # 🎯 Train CNN/LSTM/FC — freeze Transformer
                    for param in self.model.embeding_layer.parameters():
                        param.requires_grad = True
                    for param in self.model.shared_layer.parameters():
                        param.requires_grad = True
                    for param in self.model.personal_layer.parameters():
                        param.requires_grad = False
                    active_optims = [self.global_optimizer]
                    active_scheds = [self.scheduler_g]
                    phase_name = "global"
                else:
                    for param in self.model.embeding_layer.parameters():
                        param.requires_grad = False
                    for param in self.model.shared_layer.parameters():
                        param.requires_grad = False
                    for param in self.model.personal_layer.parameters():
                        param.requires_grad = True
                    active_optims = [self.personal_optimizer]
                    active_scheds = [self.scheduler_p]
                    phase_name = "personal"
                self.model.train()
                train_loss, correct_train, total_train = 0, 0, 0
                max_batches = int(train_length * (frac+start)) +1 
                start_batch = int(train_length * start)

                for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                    if batch_idx < start_batch:
                        continue
                    elif batch_idx >= max_batches:
                        break
                    
                    bt0 = time.perf_counter()
                    batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                    
                    for opt in active_optims:
                        opt.zero_grad()

                    outputs = self.model(batch_x)
                    loss = self.criterion(outputs, batch_y)
                    loss.backward()

                    for opt in active_optims:
                        opt.step()




                    # === Metrics ===
                    batch_loss = loss.item()
                    preds = (outputs >= 0.5).float()
                    batch_acc = (preds == batch_y).float().mean().item()

                    if log >= 1 and (batch_idx + 1) % (20/log) == 0:
                        print(f"\rEpoch {epoch+1}/{epochs}: Training {phase_name} | "
                            f"Batch {batch_idx+1}/{max_batches} | "
                            f"Loss: {batch_loss:.4f}, Acc: {batch_acc:.4f}", end='')
                    
                    batch_time = time.perf_counter() - bt0

                    self.batch_time.append(batch_time)


                    self.batch_train_losses.append(batch_loss)
                    self.batch_train_accs.append(batch_acc)


                if log >= 2:
                    print(f'\r total training batch size {max_batches-start_batch}'.ljust(100), end='')
                    with torch.no_grad():
                        for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                            if batch_idx < start_batch:
                                continue
                            elif batch_idx >= max_batches:
                                break
                            batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)

                            outputs = self.model(batch_x)
                            loss = self.criterion(outputs, batch_y)

                            batch_loss = loss.item()
                            preds = (outputs >= 0.5).float()
                            batch_acc = (preds == batch_y).float().mean().item()

                            train_loss += batch_loss * batch_x.size(0)
                            correct_train += (preds == batch_y).sum().item()
                            total_train += batch_x.size(0)

                        avg_train_loss = train_loss / total_train
                        train_acc = correct_train / total_train
                        self.epoch_train_losses.append(avg_train_loss)
                        self.epoch_train_accs.append(train_acc)


                    # === Validation ===
                if self.val_loader is not None:
                    avg_val_loss, val_acc = self.evaluate(val_frac)

                    
                    if log > 1:
                        print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                            f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                            f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
                        self.epoch_val_losses.append(avg_val_loss)
                        self.epoch_val_accs.append(val_acc)

                else:
                    if log > 1:
                        print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                            f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}")


                for sched in active_scheds:
                    if phase == 1 or self.val_loader is not None:
                        sched.step(round(val_acc, 4))

                epoch_time = time.perf_counter() - et0
                self.epoch_time.append(epoch_time)

                if self.val_loader is not None and avg_val_loss is not None:
                    if avg_val_loss < self.best_val_loss:
                        self.best_val_loss = avg_val_loss
                        torch.save(self.model.state_dict(), os.path.join(self.run_folder, "best_loss_model.pt"))
                        if log >= 1:
                            print(f"💾 Best Loss Model Saved! Val Loss = {avg_val_loss:.4f}")
        if self.save > 1:
            self.save_training_data()


    def evaluate(self, frac=1.0, start=0, log=2):
        self.model.eval()
        val_loss, correct_val, total_val = 0, 0, 0
        max_batches = max(int(len(self.val_loader) * (frac+start)), 0)+1
        start_batch = int(len(self.val_loader) * start)
        if log > 2:
            print('\rstarting from batch', start_batch, 'ending to batch', max_batches, f'total validation batch size {max_batches}')
        with torch.no_grad():
            for batch_idx, (batch_x, batch_y) in enumerate(self.val_loader):
                if batch_idx < start_batch:
                    continue
                elif batch_idx >= max_batches:
                    break
                batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                avg_batch_loss = loss.item()
                val_loss += avg_batch_loss * batch_x.size(0)
                preds = (outputs >= 0.5).float()
                correct_val += (preds == batch_y).sum().item()
                total_val += batch_x.size(0)
        avg_val_loss = val_loss / total_val
        val_acc = correct_val / total_val
        return avg_val_loss, val_acc

    def test(self, test_loader):
        self.model.eval()
        all_preds = []
        all_probs = []
        all_targets = []

        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x = batch_x.to(self.device)
                batch_y = batch_y.to(self.device).float().unsqueeze(1)

                outputs = self.model(batch_x)
                probs = outputs.cpu().numpy().flatten()
                preds = (outputs >= 0.5).float().cpu().numpy().flatten()
                targets = batch_y.cpu().numpy().flatten()

                all_probs.extend(probs)
                all_preds.extend(preds)
                all_targets.extend(targets)

        # Convert to arrays
        all_probs = np.array(all_probs)
        all_preds = np.array(all_preds)
        all_targets = np.array(all_targets)

        # Metrics
        try:
            roc = roc_auc_score(all_targets, all_probs)
        except:
            roc = None

        metrics = {
            "accuracy": float(accuracy_score(all_targets, all_preds)),
            "precision": float(precision_score(all_targets, all_preds, zero_division=0)),
            "recall": float(recall_score(all_targets, all_preds, zero_division=0)),
            "f1": float(f1_score(all_targets, all_preds, zero_division=0)),
            "roc_auc": None if roc is None else float(roc),
            "r2_score": float(r2_score(all_targets, all_probs)),
            "classification_report": classification_report(all_targets, all_preds, output_dict=True),
            "confusion_matrix": confusion_matrix(all_targets, all_preds).tolist()
        }

        # Save to JSON
        save_path = os.path.join(self.run_folder, "test_metrics.json")
        with open(save_path, "w") as f:
            json.dump(metrics, f, indent=4)

        print(f"✅ Test metrics saved to: {save_path}")
        return metrics

    def _create_run_folder(self, folder_name="run"):
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        self.main_dir = "training_runs"
        os.makedirs(self.main_dir, exist_ok=True)

        self.run_folder = os.path.join(self.main_dir, f"{timestamp}_{folder_name}")
        os.makedirs(self.run_folder, exist_ok=True)

        print("Run folder created at:", self.run_folder)

    def save_training_data(self):
        data = {
            "batch_train_losses": self.batch_train_losses,
            "batch_train_accs": self.batch_train_accs,
            "epoch_train_losses": self.epoch_train_losses,
            "epoch_train_accs": self.epoch_train_accs,
            "epoch_val_losses": self.epoch_val_losses,
            "epoch_val_accs": self.epoch_val_accs,
            "batch_times": self.batch_time,
            "epoch_times": self.epoch_time
        }


        # Save as JSON
        with open(os.path.join(self.run_folder, "logs.json"), "w") as f:
            json.dump(data, f, indent=4)

    



In [ ]:
from plot_graph import plot_run
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 🔧 Training Config
# ============================================================
num_epochs = [4,0,0]
lr_g = 0.001
lr_p = 0.001

# Store all dataset metrics
all_results = {}

# ============================================================
# 🔁 Training Loop for Each Dataset
# ============================================================
nn_model = {}
for dataset_name, loaders in dataloader_dict.items():
    print("\n" + "="*70)
    print(f"🚀 Training model on {dataset_name.upper()} dataset")
    print("="*70)
    for frac in  [1]:
        print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))
        train_loader = loaders["train_loader"]
        val_loader = loaders["val_loader"]


        

        # Initialize model, loss, optimizer
        nn_model[dataset_name] = URLBinaryCNN(vocab_size=len(vocab), embed_dim=64, maxlen = 60).to(device)
        global_params = []
        persnalization_params = []
        for name, param in nn_model[dataset_name].embeding_layer.named_parameters():
            global_params.append(param)
        for name, param in nn_model[dataset_name].shared_layer.named_parameters():
            global_params.append(param)
        for name, param in nn_model[dataset_name].personal_layer.named_parameters():
            persnalization_params.append(param)
        criterion = nn.BCELoss()
        personal_optimizer = torch.optim.NAdam(global_params, lr=lr_p, weight_decay=lr_p/10)
        global_optimizer = torch.optim.NAdam(persnalization_params, lr=lr_g, weight_decay=lr_g/10)
        personal_scheduler = optim.lr_scheduler.ReduceLROnPlateau(personal_optimizer, mode='max', factor=0.5, patience=3)
        global_scheduler = optim.lr_scheduler.ReduceLROnPlateau(global_optimizer, mode='max', factor=0.5, patience=3)
        run_name='depthwisecnn(64,64)[3] bilstm(64,64) size with layer normalization'
        trainer = Train(nn_model[dataset_name], criterion,run_name=run_name, train_loader=train_loader, val_loader=val_loader, personal_optimizer=personal_optimizer, global_optimizer=global_optimizer, scheduler_g=global_scheduler, scheduler_p=personal_scheduler, save=2)
        # Lists to track performance
        trainer.train(num_epochs,frac=frac,val_frac=frac, log=2)
        trainer.test(loaders['test_loader'])
        plot_run(trainer.run_folder)
    

    # Store all results for this dataset
    all_results[dataset_name] = {
        "batch_train_losses": trainer.batch_train_losses,
        "batch_train_accs": trainer.batch_train_accs,
        "epoch_train_losses": trainer.epoch_train_losses,
        "epoch_train_accs": trainer.epoch_train_accs,
        "epoch_val_losses": trainer.epoch_val_losses,
        "epoch_val_accs": trainer.epoch_val_accs,
        "batch_times": trainer.batch_time,
        "epoch_times": trainer.epoch_time
    }

print("\n All datasets trained successfully!")

# ============================================================
# Summary of All Results
# ============================================================
print("\n" + "="*70)
print("Final Validation Accuracy Summary")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res["epoch_val_losses"][-1]:.4f} | Val Loss: {res["epoch_val_accs"][-1]:.4f}")



🚀 Training model on DATASET 4 (KAGGELS/TARUNTIWARIHP/PHISHING-SITE-URLS) dataset
__________🧩 Using 100% of training data___________
Run folder created at: training_runs\2025-11-20_08-26-39_depthwisecnn(64,64)[3] bilstm(64,64) size with layer normalization
📁 Model configuration saved at: training_runs\2025-11-20_08-26-39_depthwisecnn(64,64)[3] bilstm(64,64) size with layer normalization\model_configuration.json
Epoch 1/4 Training global+personal| Train Loss: 0.1397, Train Acc: 0.9496 | Val Loss: 0.1336, Val Acc: 0.9503
💾 Best Loss Model Saved! Val Loss = 0.1336
Epoch 2/4 Training global+personal| Train Loss: 0.1004, Train Acc: 0.9651 | Val Loss: 0.0986, Val Acc: 0.9642
💾 Best Loss Model Saved! Val Loss = 0.0986
Epoch 3/4 Training global+personal| Train Loss: 0.0839, Train Acc: 0.9722 | Val Loss: 0.0859, Val Acc: 0.9696
💾 Best Loss Model Saved! Val Loss = 0.0859
Epoch 4/4 Training global+personal| Train Loss: 0.1036, Train Acc: 0.9651 | Val Loss: 0.1100, Val Acc: 0.9618
✅ Test metrics s

In [43]:
print(loaders.keys())

dict_keys(['train_loader', 'val_loader', 'test_loader'])


In [35]:
plot_run(r"training_runs\only cnn with kernal 3 outdimention 64_2025-11-20_02-38-01")

✅ All graphs saved in: training_runs\only cnn with kernal 3 outdimention 64_2025-11-20_02-38-01\graphs


In [30]:
#cross dataset accuracy
print("train_datasets".ljust(50,' ')+"| test_datasets".ljust(150,' '))
print(' '*50, end='')
for test_datset_name in nn_model:
    print(f"| {test_datset_name}".ljust(50,' '), end='')
print()
print('='+(('='*49)+'|')*4)
for train_dataset_name in nn_model:
    print(f'{train_dataset_name}'.ljust(50,' '), end='')
    for test_dataset_name in nn_model.keys():
        train_loader = dataloader_dict[test_dataset_name]['train_loader']
        test_loader = dataloader_dict[test_dataset_name]['test_loader']
        trainer = Train(nn_model[train_dataset_name], criterion, train_loader=train_loader, val_loader=test_loader)
        avg_val_loss, val_acc = trainer.evaluate(log=0)   
        print(f"| {avg_val_loss:.4f},{val_acc:.4f}".ljust(50,' '), end='')
    print()


train_datasets                                    | test_datasets                                                                                                                                       
                                                  | Dataset 1 (Malicious URLs)                      | Dataset 2 (ndarvind/phiusiil-phishing)          | Dataset 3 (kmack/Phishing_urls)                 
==================================================|=================================================|=================================================|=================================================|
Dataset 1 (Malicious URLs)                        | 0.0773,0.9748                                   | 3.0455,0.5779                                   | 2.2636,0.5212                                   
Dataset 2 (ndarvind/phiusiil-phishing)            | 1.2805,0.8101                                   | 0.1402,0.9974                                   | 3.9363,0.4883                              

In [25]:
nn_model.keys()

dict_keys(['Dataset 1 (Malicious URLs)', 'Dataset 2 (ndarvind/phiusiil-phishing)', 'Dataset 3 (kmack/Phishing_urls)'])

In [ ]:
nn_model[dataset_name]
features, labels = [], []

# 🔹 Extract CNN features for training set
with torch.no_grad():
    for x_batch, y_batch in tqdm(train_loader, desc="🔍 Extracting Train Features", unit="batch"):
        x_batch = x_batch.to(device, non_blocking=True)
        feats = nn_model[dataset_name].extract_features(x_batch)
        features.append(feats.cpu().numpy())
        labels.append(y_batch.cpu().numpy())

X_train = np.concatenate(features, axis=0)
y_train = np.concatenate(labels, axis=0)

# Free memory before val extraction
del features, labels, x_batch, y_batch, feats
torch.cuda.empty_cache()

# 🔹 Extract CNN features for validation set
features, labels = [], []
with torch.no_grad():
    for x_batch, y_batch in tqdm(val_loader, desc="🔍 Extracting Val Features", unit="batch"):
        x_batch = x_batch.to(device, non_blocking=True)
        feats = nn_model[dataset_name].extract_features(x_batch)
        features.append(feats.cpu().numpy())
        labels.append(y_batch.cpu().numpy())

X_val = np.concatenate(features, axis=0)
y_val = np.concatenate(labels, axis=0)


🔍 Extracting Train Features:   0%|          | 0/798 [00:00<?, ?batch/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (31440x128 and 7680x64)

In [87]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.02,
    eval_metric="logloss",
    max_depth = 10,
    random_state=42
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)


[0]	validation_0-logloss:0.66290
[1]	validation_0-logloss:0.64412
[2]	validation_0-logloss:0.62610
[3]	validation_0-logloss:0.60878
[4]	validation_0-logloss:0.59213
[5]	validation_0-logloss:0.57611
[6]	validation_0-logloss:0.56067
[7]	validation_0-logloss:0.54580
[8]	validation_0-logloss:0.53145
[9]	validation_0-logloss:0.51761
[10]	validation_0-logloss:0.50425
[11]	validation_0-logloss:0.49134
[12]	validation_0-logloss:0.47886
[13]	validation_0-logloss:0.46680
[14]	validation_0-logloss:0.45513
[15]	validation_0-logloss:0.44383
[16]	validation_0-logloss:0.43289
[17]	validation_0-logloss:0.42230
[18]	validation_0-logloss:0.41203
[19]	validation_0-logloss:0.40208
[20]	validation_0-logloss:0.39244
[21]	validation_0-logloss:0.38308
[22]	validation_0-logloss:0.37400
[23]	validation_0-logloss:0.36518
[24]	validation_0-logloss:0.35663
[25]	validation_0-logloss:0.34832
[26]	validation_0-logloss:0.34025
[27]	validation_0-logloss:0.33240
[28]	validation_0-logloss:0.32478
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, ...)

In [88]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

y_pred_prob = xgb_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_pred_prob))


Accuracy: 0.9968560139355058
Precision: 0.9950535252860834
Recall: 0.9994809047089358
F1: 0.9972623011468739
ROC AUC: 0.9964077822390681


In [27]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(
    n_estimators=100,   
    max_depth=30,     
    random_state=42
)



rfc_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=30, random_state=42)

In [28]:
y_pred_prob = rfc_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_pred_prob))

Accuracy: 0.9119705508089438
Precision: 0.8970665575877987
Recall: 0.9306477524173259
F1: 0.9135486558459913
ROC AUC: 0.911979035046978


In [ ]:
import pickle
f= open('tld_encoding_serise_dataset_1.bin','rb')
tld_stats = pickle.load(file=f)
f.close()
print(type(tld_stats))

import kagglehub
import os
# Download latest version
folder_path = kagglehub.dataset_download("cheedcheed/top1m")

print("Path to dataset files:", folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

# read the first CSV file
file_path = os.path.join(folder_path, csv_files[0])
alexa_top_1m_domain = pd.read_csv(file_path,header=None,names=['rank', 'domain'])
alexa_domains_set = set(alexa_top_1m_domain['domain'].apply(str.lower))

# --- Helper function: Shannon entropy ---
def safe_parse(url: str):
    """Safely parse URLs, adding http:// if missing and handling bad IPv6 parts."""
    if not isinstance(url, str) or not url.strip():
        return urlparse("http://")  

    # Ensure scheme exists
    if not re.match(r'^[a-zA-Z]+://', url):
        url = 'http://' + url

    # Clean invalid brackets that trigger IPv6 errors
    url = re.sub(r'\[.*?\]', '', url)

    try:
        return urlparse(url)
    except ValueError:
        # fallback: strip more aggressively if still malformed
        url = re.sub(r'[^a-zA-Z0-9:/._\-?&=]', '', url)
        return urlparse(url)
def calculate_entropy(string):
    """Measures randomness of characters in the URL."""
    if not string:
        return 0
    freq = {char: string.count(char) for char in set(string)}
    entropy = -sum((count / len(string)) * math.log2(count / len(string)) for count in freq.values())
    return entropy

# --- Main feature extraction function ---
def extract_handcrafted_features(url):
    features = {}
    if not re.match(r'^[hH]+[tT]+[tT]+[pP]+[sS]+://', url):
        url = 'http://' + url
    parsed = safe_parse(url)
    
    # 1️⃣ Basic structural features
    features['url_length'] = len(url)
    features['hostname_length'] = len(parsed.netloc)
    features['path_length'] = len(parsed.path)
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['num_digits'] = sum(c.isdigit() for c in url)
    features['num_letters'] = sum(c.isalpha() for c in url)
    features['num_params'] = url.count('?')
    features['num_equals'] = url.count('=')
    features['num_slashes'] = url.count('/')
    features['num_at'] = url.count('@')

    # 2️⃣ Lexical / composition cues
    features['has_https'] = 1 if url.lower().startswith('https') else 0
    features['has_ip'] = 1 if re.search(r'(\d{1,3}\.){3}\d{1,3}', parsed.netloc) else 0
    features['has_subdomain'] = 1 if parsed.netloc.count('.') > 1 else 0
    features['has_suspicious_words'] = 1 if re.search(r'(login|secure|verify|update|free|bank|click)', url.lower()) else 0

    # 3️⃣ Domain / TLD features
    extracted = tldextract.extract(url)
    main_domain = f"{extracted.domain}.{extracted.suffix}"
    if ':' in main_domain:  # remove port
        main_domain = main_domain.split(':')[0]
    features['domain_length'] = len(main_domain)
    features['in_alexa_top1m'] = 1 if main_domain in alexa_domains_set else 0
    '''
    if features['in_alexa_top1m'] == 0 and main_domain:  # only check if domain not in top1M
        # find closest match in Alexa domains
        best_match, score, _ = process.extractOne(main_domain, alexa_domains_set, scorer=fuzz.ratio)
        features['closest_alexa_domain'] = best_match
        features['closest_alexa_score'] = score  # 0-100
    else:
        features['closest_alexa_score'] = 1000  # high score to show that it is original url
    '''
    ext = tldextract.extract(url)
    tld = ext.suffix    # "com", "co.uk", "org"
    features['tld'] = tld if tld else 'unknown'
    features['tld_phish_ratio'] = tld_stats['phish_ratio'].get(features['tld'], 0.5)
    features['tld_total_frequency'] = tld_stats['total'].get(features['tld'], 1)

    # 4️⃣ Ratios
    features['digit_ratio'] = features['num_digits'] / (features['url_length'] + 1e-5)
    features['special_char_ratio'] = (features['num_hyphens'] + features['num_dots'] + features['num_slashes']) / (features['url_length'] + 1e-5)

    # 5️⃣ Entropy (measures randomness / obfuscation)
    features['url_entropy'] = calculate_entropy(url)

    # 6️⃣ Misplacement indicators
    # '@' symbol used to hide real domain (like "http://evil.com@legit.com")
    features['at_in_domain'] = 1 if '@' in parsed.netloc else 0
    
    # Double slashes '//' appearing after path (used to trick users)
    features['double_slash_in_path'] = 1 if re.search(r'/.+//', parsed.path) else 0

    return features

<class 'pandas.core.frame.DataFrame'>
Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\cheedcheed\top1m\versions\1


In [92]:
from tqdm import tqdm
tqdm.pandas()
for i in encoded_data:

    for split in encoded_data[i]:
        print("feature extracting", i, split)
        encoded_data[i][split] = encoded_data[i][split].assign(**encoded_data[i][split].url.progress_apply(lambda url : pd.Series(extract_handcrafted_features(url))))

feature extracting Dataset 2 (ndarvind/phiusiil-phishing) train


100%|██████████| 188296/188296 [00:43<00:00, 4358.87it/s]


feature extracting Dataset 2 (ndarvind/phiusiil-phishing) valid


100%|██████████| 23537/23537 [00:04<00:00, 5001.43it/s]


feature extracting Dataset 2 (ndarvind/phiusiil-phishing) test


100%|██████████| 23537/23537 [00:05<00:00, 4503.97it/s]


In [96]:
# Split features and labels
independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependet_features  = 'label'
model_dict = {}
for i in encoded_data:
    print(f"train for {i}")
    X_train = encoded_data[i]['train'][independent_features]
    y_train = encoded_data[i]['train'][dependet_features]
    X_test = encoded_data[i]['valid'][independent_features]
    y_test = encoded_data[i]['valid'][dependet_features]

    # Create XGBoost classifier
    model_dict[i] = xgb.XGBClassifier(
        n_estimators=100,      # number of boosting rounds
        learning_rate=0.01,     # step size shrinkage
        max_depth=30,           # tree depth
        eval_metric='logloss', # evaluation metric
        use_label_encoder=False
    )

    # Train the model
    model_dict[i].fit(X_train, y_train)
    # Predict
    y_pred = model_dict[i] .predict(X_test)

    # Evaluate
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall   :", recall_score(y_test, y_pred))
    print("F1-score :", f1_score(y_test, y_pred))
    print("ROC AUC  :", roc_auc_score(y_test, y_pred))



train for Dataset 2 (ndarvind/phiusiil-phishing)


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:01:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     10052
           1       1.00      1.00      1.00     13485

    accuracy                           1.00     23537
   macro avg       1.00      1.00      1.00     23537
weighted avg       1.00      1.00      1.00     23537


📗 VALIDATION METRICS
Accuracy : 0.9973233632153631
Precision: 0.9954961606615476
Recall   : 0.999851687059696
F1-score : 0.997669170150579
ROC AUC  : 0.9968916214844837


In [95]:
#logistic_regression(nn+xgboost)
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependent_features  = 'label'
logistic_model = {}
for name in encoded_data:
    #print(model_dict[name])
    #print(nn_model[name])
    nn_model[name].eval()
    with torch.no_grad():
        model = nn_model[name]
        #print(model)
        x_ = encoded_data[name]['train']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_train = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_train.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_train = torch.cat(nn_preds_train, dim=0).cpu()

    # XGBoost predictions
    xgb_preds_train = model_dict[name].predict_proba(encoded_data[name]['train'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X = np.column_stack((nn_preds_train, xgb_preds_train))
    meta_y = encoded_data[name]['train'][dependent_features]


    meta_model= LogisticRegressionCV(
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    penalty="l2",
    scoring="roc_auc",
    solver="lbfgs",
    Cs=10,
    max_iter=1000,
    n_jobs=-1
)
    meta_model.fit(meta_X, meta_y)


    with torch.no_grad():
        x_ = encoded_data[name]['valid']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_val = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_val.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_val = torch.cat(nn_preds_val, dim=0).cpu()
    # XGBoost predictions
    xgb_preds_val = model_dict[name].predict_proba(encoded_data[name]['valid'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X_val = np.column_stack((nn_preds_val, xgb_preds_val))
    meta_y_val = encoded_data[name]['valid'][dependent_features]


    y_pred_train = meta_model.predict(meta_X)
    y_pred_prob_train = meta_model.predict_proba(meta_X)[:, 1]

    print("\n📘 TRAIN METRICS")
    print("Accuracy :", accuracy_score(meta_y, y_pred_train))
    print("Precision:", precision_score(meta_y, y_pred_train))
    print("Recall   :", recall_score(meta_y, y_pred_train))
    print("F1-score :", f1_score(meta_y, y_pred_train))
    print("ROC AUC  :", roc_auc_score(meta_y, y_pred_prob_train))

    # ===============================
    # 🔹 Validation Metrics
    # ===============================
    y_pred_val = meta_model.predict(meta_X_val)
    y_pred_prob_val = meta_model.predict_proba(meta_X_val)[:, 1]

    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(meta_y_val, y_pred_val))
    print("Precision:", precision_score(meta_y_val, y_pred_val))
    print("Recall   :", recall_score(meta_y_val, y_pred_val))
    print("F1-score :", f1_score(meta_y_val, y_pred_val))
    print("ROC AUC  :", roc_auc_score(meta_y_val, y_pred_prob_val))



Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 368/368 [00:07
Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 46/46 [00:00<0



📘 TRAIN METRICS
Accuracy : 0.998231507838722
Precision: 0.9970237820151399
Recall   : 0.999898034853541
F1-score : 0.9984588399183602
ROC AUC  : 0.9991025245037448

📗 VALIDATION METRICS
Accuracy : 0.9977057398988826
Precision: 0.9961581086073144
Recall   : 0.999851687059696
F1-score : 0.9980014803849001
ROC AUC  : 0.9991928475450093


In [97]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependent_features  = 'label'
for name in encoded_data:
    #print(model_dict[name])
    #print(nn_model[name])
    nn_model[name].eval()
    with torch.no_grad():
        model = nn_model[name]
        #print(model)
        x_ = encoded_data[name]['train']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_train = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_train.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_train = torch.cat(nn_preds_train, dim=0).cpu()

    # XGBoost predictions
    xgb_preds_train = model_dict[name].predict_proba(encoded_data[name]['train'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X = np.column_stack((nn_preds_train, xgb_preds_train))
    meta_y = encoded_data[name]['train'][dependent_features]


    params = {
    "hidden_layer_sizes": [(8,), (16,), (24,)],
    "alpha": [1e-3, 1e-4, 1e-5],  # L2 regularization
    "learning_rate_init": [1e-3],
    "early_stopping": [True],
    "max_iter": [500],
    }

    mlp = MLPClassifier(random_state=42)
    meta_model = GridSearchCV(mlp, params, cv=StratifiedKFold(3), scoring="accuracy", n_jobs=-1, verbose=3)
    meta_model.fit(meta_X, meta_y)


    with torch.no_grad():
        x_ = encoded_data[name]['valid']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_val = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_val.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_val = torch.cat(nn_preds_val, dim=0).cpu()
    # XGBoost predictions
    xgb_preds_val = model_dict[name].predict_proba(encoded_data[name]['valid'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X_val = np.column_stack((nn_preds_val, xgb_preds_val))
    meta_y_val = encoded_data[name]['valid'][dependent_features]


    y_pred_train = meta_model.predict(meta_X)
    y_pred_prob_train = meta_model.predict_proba(meta_X)[:, 1]

    print("\n📘 TRAIN METRICS")
    print("Accuracy :", accuracy_score(meta_y, y_pred_train))
    print("Precision:", precision_score(meta_y, y_pred_train))
    print("Recall   :", recall_score(meta_y, y_pred_train))
    print("F1-score :", f1_score(meta_y, y_pred_train))
    print("ROC AUC  :", roc_auc_score(meta_y, y_pred_prob_train))

    # ===============================
    # 🔹 Validation Metrics
    # ===============================
    y_pred_val = meta_model.predict(meta_X_val)
    y_pred_prob_val = meta_model.predict_proba(meta_X_val)[:, 1]

    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(meta_y_val, y_pred_val))
    print("Precision:", precision_score(meta_y_val, y_pred_val))
    print("Recall   :", recall_score(meta_y_val, y_pred_val))
    print("F1-score :", f1_score(meta_y_val, y_pred_val))
    print("ROC AUC  :", roc_auc_score(meta_y_val, y_pred_prob_val))



Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 368/368 [00:07


Fitting 3 folds for each of 9 candidates, totalling 27 fits


Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 46/46 [00:00<0



📘 TRAIN METRICS
Accuracy : 0.998231507838722
Precision: 0.9970789155011601
Recall   : 0.999842417500927
F1-score : 0.9984587543217361
ROC AUC  : 0.9991044450185509

📗 VALIDATION METRICS
Accuracy : 0.9977057398988826
Precision: 0.9962314342717801
Recall   : 0.9997775305895439
F1-score : 0.9980013324450366
ROC AUC  : 0.9991988231459664


In [98]:
#xgboost(nn+xgboost)
from sklearn.linear_model import LogisticRegression

independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependent_features  = 'label'
logistic_model = {}
for name in encoded_data:
    #print(model_dict[name])
    #print(nn_model[name])
    nn_model[name].eval()
    with torch.no_grad():
        model = nn_model[name]
        #print(model)
        x_ = encoded_data[name]['train']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_train = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_train.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_train = torch.cat(nn_preds_train, dim=0).cpu()

    # XGBoost predictions
    xgb_preds_train = model_dict[name].predict_proba(encoded_data[name]['train'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X = np.column_stack((nn_preds_train, xgb_preds_train))
    meta_y = encoded_data[name]['train'][dependent_features]

    with torch.no_grad():
        x_ = encoded_data[name]['valid']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_val = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_val.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_val = torch.cat(nn_preds_val, dim=0).cpu()
    # XGBoost predictions
    xgb_preds_val = model_dict[name].predict_proba(encoded_data[name]['valid'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X_val = np.column_stack((nn_preds_val, xgb_preds_val))
    meta_y_val = encoded_data[name]['valid'][dependent_features]


    xgb_model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.01,
        bjective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        max_depth = 10,
        random_state=42,
        
    )

    xgb_model.fit(
        meta_X, meta_y,
        #eval_set=[(meta_X_val, meta_y_val)],
    )


    y_pred_train = meta_model.predict(meta_X)
    y_pred_prob_train = meta_model.predict_proba(meta_X)[:, 1]

    print("\n📘 TRAIN METRICS")
    print("Accuracy :", accuracy_score(meta_y, y_pred_train))
    print("Precision:", precision_score(meta_y, y_pred_train))
    print("Recall   :", recall_score(meta_y, y_pred_train))
    print("F1-score :", f1_score(meta_y, y_pred_train))
    print("ROC AUC  :", roc_auc_score(meta_y, y_pred_prob_train))

    # ===============================
    # 🔹 Validation Metrics
    # ===============================
    y_pred_val = meta_model.predict(meta_X_val)
    y_pred_prob_val = meta_model.predict_proba(meta_X_val)[:, 1]

    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(meta_y_val, y_pred_val))
    print("Precision:", precision_score(meta_y_val, y_pred_val))
    print("Recall   :", recall_score(meta_y_val, y_pred_val))
    print("F1-score :", f1_score(meta_y_val, y_pred_val))
    print("ROC AUC  :", roc_auc_score(meta_y_val, y_pred_prob_val))


Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 368/368 [00:07
Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 46/46 [00:00<0
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:02:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "bjective", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📘 TRAIN METRICS
Accuracy : 0.998231507838722
Precision: 0.9970789155011601
Recall   : 0.999842417500927
F1-score : 0.9984587543217361
ROC AUC  : 0.9991044450185509

📗 VALIDATION METRICS
Accuracy : 0.9977057398988826
Precision: 0.9962314342717801
Recall   : 0.9997775305895439
F1-score : 0.9980013324450366
ROC AUC  : 0.9991988231459664


# Fedrated learning

In [109]:
#train global model on little part(5%) of dataset
dataset_name = 'Dataset 2 (ndarvind/phiusiil-phishing)'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
global_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
loaders = dataloader_dict[dataset_name]
train_loader = loaders["train_loader"]
val_loader = loaders["val_loader"]
lr  = 0.01
print(f"🧩 Using {0.05*100:.0f}% of training data".center(50, '_'))

criterion = nn.BCELoss()
optimizer = torch.optim.NAdam(global_model.parameters(), lr=lr, weight_decay=lr/10)
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
# Lists to track performance
trainer.train(epochs_list=[5],frac=0.05,val_frac=0.05, log=2)



___________🧩 Using 5% of training data____________
Epoch 1/5 Training CNN + Embeding| Train Loss: 0.5441, Train Acc: 0.7316 | Val Loss: 0.5311, Val Acc: 0.7398
Epoch 2/5 Training CNN + Embeding| Train Loss: 0.4938, Train Acc: 0.7921 | Val Loss: 0.4803, Val Acc: 0.8094
Epoch 3/5 Training CNN + Embeding| Train Loss: 0.5178, Train Acc: 0.7539 | Val Loss: 0.5223, Val Acc: 0.7461
Epoch 4/5 Training CNN + Embeding| Train Loss: 0.4943, Train Acc: 0.7724 | Val Loss: 0.4927, Val Acc: 0.7719
Epoch 5/5 Training CNN + Embeding| Train Loss: 0.2988, Train Acc: 0.9098 | Val Loss: 0.3008, Val Acc: 0.9031


In [103]:
import numpy as np
# using alpha to produce unidentical splits high alpha more identical
num_clients = 5
alpha = 0.5
total_data = 0.5
client_fractions = np.random.dirichlet([alpha] * num_clients) * total_data
print(client_fractions)
print(client_fractions.sum())


[1.50460524e-04 3.88989434e-01 1.17394893e-02 3.28685992e-02
 6.62520174e-02]
0.5


In [104]:
client_models = []
for i in range(num_clients):
    client_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)                     # fresh instance
    client_model.load_state_dict(global_model.state_dict())
    client_models.append(client_model)

In [105]:
print('clints initial accuracy')
start = 0
for i, model in enumerate(client_models):
    criterion = nn.BCELoss()
    trainer = Train(model, criterion=criterion, train_loader=train_loader, val_loader=val_loader)
    avg_val_loss, val_acc = trainer.evaluate(frac=client_fractions[i], start=start, log=2)
    start += client_fractions[i]
    print(avg_val_loss, val_acc)

clints initial accuracy
0.00031488906824961305 1.0
0.016156392587112753 0.9976128472222222
0.00047816982259973884 1.0
0.014619543587156971 0.9977678571428571
0.014312390919907816 0.9977678571428571


In [106]:
# trainig clints
start = 0
for i, model in enumerate(client_models):
    criterion = nn.BCELoss()
    trainer = Train(model, criterion=criterion, train_loader=train_loader, val_loader=val_loader)
    trainer.train(epochs_list=[2,0,0],frac=client_fractions[i], val_frac=client_fractions[i], start=start, log=2)
    start += client_fractions[i]


Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0997, Train Acc: 0.9922 | Val Loss: 0.0084, Val Acc: 0.9922
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0012, Train Acc: 1.0000 | Val Loss: 0.0079, Val Acc: 0.9922
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0177, Train Acc: 0.9973 | Val Loss: 0.0144, Val Acc: 0.9977
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.4952, Train Acc: 0.7842 | Val Loss: 0.4883, Val Acc: 0.8171
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0370, Train Acc: 0.9971 | Val Loss: 0.0294, Val Acc: 0.9948
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0056, Train Acc: 0.9996 | Val Loss: 0.0176, Val Acc: 0.9974
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0171, Train Acc: 0.9981 | Val Loss: 0.0152, Val Acc: 0.9978
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0196, Train Acc: 0.9973 | Val Loss: 0.0155, Val Acc: 0.9978
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0243, Train Acc: 0.9968 | Val Loss: 0.0138, Val Acc: 0.9982
Epoch 2/2 Training 

In [107]:
# tansfering weights and averaging them
import copy
client_weights = [client_model.state_dict() for client_model in client_models]
total_samples = sum(client_fractions)

new_global_state = copy.deepcopy(global_model.state_dict())

# set all params to zero before summing
for key in new_global_state.keys():
    new_global_state[key] = torch.zeros_like(new_global_state[key])

# aggregate client updates
for client_model, n_i in zip(client_models, client_fractions):
    client_state = client_model.state_dict()
    for key in new_global_state.keys():
        new_global_state[key] += client_state[key] * (n_i / total_samples)

# load averaged weights back into global model
global_model.load_state_dict(new_global_state)



<All keys matched successfully>

In [108]:
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
avg_val_loss, val_acc = trainer.evaluate(frac=0.3, start=0, log=2)
print(avg_val_loss, val_acc)

0.5458191404385226 0.6873604910714286


In [110]:
import copy
def make_clients(num_clients=5, alpha=0.5, total_data=1):
    import numpy as np
    # using alpha to produce unidentical splits high alpha more identical
    client_fractions = np.random.dirichlet([alpha] * num_clients) * total_data
    client_models = []
    for i in range(num_clients):
        client_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)                     # fresh instance
        client_model.load_state_dict(global_model.state_dict())
        client_models.append(client_model)
    return client_model, client_fractions
def train_client(client_models, client_fractions, start=0):
    for i, _ in enumerate(client_models):
        criterion = nn.BCELoss()
        trainer = Train(client_models[i], criterion=criterion, train_loader=train_loader, val_loader=val_loader)
        trainer.train(epochs_list=[2,0,0],frac=client_fractions[i], val_frac=client_fractions[i], start=start, log=2)
        start += client_fractions[i]
def fed_avg(client_models, client_fractions):
    total_samples = sum(client_fractions)
    new_global_state = copy.deepcopy(global_model.state_dict())
    # set all params to zero before summing
    for key in new_global_state.keys():
        new_global_state[key] = torch.zeros_like(new_global_state[key])
    # aggregate client updates
    for client_model, n_i in zip(client_models, client_fractions):
        client_state = client_model.state_dict()
        for key in new_global_state.keys():
            new_global_state[key] += client_state[key] * (n_i / total_samples)
    # load averaged weights back into global model
    global_model.load_state_dict(new_global_state)
def update_clients(client_models):
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())
client_model, client_fractions = make_clients(5, total_data=0.3) 

In [111]:
for i in range(3):
    train_client(client_models, client_fractions, start=0.3)   
    fed_avg(client_models, client_fractions)
    update_clients(client_models)
    trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
    avg_val_loss, val_acc = trainer.evaluate(frac=0.3, start=0, log=2)
    print(f'global model loss {avg_val_loss} accuracy {val_acc}')

Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0146, Train Acc: 0.9978 | Val Loss: 0.0116, Val Acc: 0.9984
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0144, Train Acc: 0.9977 | Val Loss: 0.0105, Val Acc: 0.9984
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0250, Train Acc: 0.9971 | Val Loss: 0.0110, Val Acc: 0.9984
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0199, Train Acc: 0.9975 | Val Loss: 0.0114, Val Acc: 0.9984
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0172, Train Acc: 0.9980 | Val Loss: 0.0151, Val Acc: 0.9979
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0202, Train Acc: 0.9967 | Val Loss: 0.0130, Val Acc: 0.9979
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0178, Train Acc: 0.9980 | Val Loss: 0.0266, Val Acc: 0.9961
Epoch 2/2 Training CNN + Embeding| Train Loss: 0.0158, Train Acc: 0.9980 | Val Loss: 0.0261, Val Acc: 0.9961
Epoch 1/2 Training CNN + Embeding| Train Loss: 0.0147, Train Acc: 0.9980 | Val Loss: 0.0133, Val Acc: 0.9980
Epoch 2/2 Training 

## fedrated learning with persnalization
# meta learing



In [10]:
list(dataloader_dict.keys())[0]

'Dataset 1 (Malicious URLs)'

In [11]:
#train global model on little part(5%) of dataset
dataset_name = list(dataloader_dict.keys())[0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
global_model = URLBinaryCNN(vocab_size=len(vocab), maxlen=60).to(device)
loaders = dataloader_dict[dataset_name]
train_loader = loaders["train_loader"]
val_loader = loaders["val_loader"]
lr_p=0.001
lr_g=0.001
frac = 0.1
print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))

global_params = []
persnalization_params = []
for name, param in global_model.embeding_layer.named_parameters():
    global_params.append(param)
for name, param in global_model.shared_layer.named_parameters():
    global_params.append(param)
for name, param in global_model.personal_layer.named_parameters():
    persnalization_params.append(param)
criterion = nn.BCELoss()
personal_optimizer = torch.optim.NAdam(global_params, lr=lr_p, weight_decay=lr_p/10)
global_optimizer = torch.optim.NAdam(persnalization_params, lr=lr_g, weight_decay=lr_g/10)
personal_scheduler = optim.lr_scheduler.ReduceLROnPlateau(personal_optimizer, mode='max', factor=0.5, patience=3)
global_scheduler = optim.lr_scheduler.ReduceLROnPlateau(global_optimizer, mode='max', factor=0.5, patience=3)

trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader, personal_optimizer=personal_optimizer, global_optimizer=global_optimizer, scheduler_g=global_scheduler, scheduler_p=personal_scheduler)
        
# Lists to track performance
trainer.train(epochs_list=[10,0,0],frac=frac,val_frac=frac, log=2)



___________🧩 Using 10% of training data___________
Epoch 1/10 Training global+personal| Train Loss: 0.2040, Train Acc: 0.9251 | Val Loss: 0.1872, Val Acc: 0.9379
Epoch 2/10 Training global+personal| Train Loss: 0.1480, Train Acc: 0.9555 | Val Loss: 0.1321, Val Acc: 0.9584
Epoch 3/10 Training global+personal| Train Loss: 0.1322, Train Acc: 0.9610 | Val Loss: 0.1214, Val Acc: 0.9627
Epoch 4/10 Training global+personal| Train Loss: 0.1494, Train Acc: 0.9509 | Val Loss: 0.1381, Val Acc: 0.9523
Epoch 5/10 Training global+personal| Train Loss: 0.1142, Train Acc: 0.9641 | Val Loss: 0.1058, Val Acc: 0.9641
Epoch 6/10 Training global+personal| Train Loss: 0.1112, Train Acc: 0.9649 | Val Loss: 0.0984, Val Acc: 0.9662
Epoch 7/10 Training global+personal| Train Loss: 0.1138, Train Acc: 0.9626 | Val Loss: 0.1057, Val Acc: 0.9618
Epoch 8/10 Training global+personal| Train Loss: 0.1051, Train Acc: 0.9674 | Val Loss: 0.0973, Val Acc: 0.9681
Epoch 9/10 Training global+personal| Train Loss: 0.1018, Trai

In [12]:
import copy
def make_clients(num_clients=5, alpha=0.5, total_data=1):
    import numpy as np
    # using alpha to produce unidentical splits high alpha more identical
    client_fractions = np.random.dirichlet([alpha] * num_clients) * total_data
    client_models = []
    for i in range(num_clients):
        client_model = URLBinaryCNN(vocab_size=len(vocab), maxlen=60).to(device)                     # fresh instance
        client_model.load_state_dict(global_model.state_dict())
        client_models.append(client_model)
    return client_models, client_fractions
def quick_fine_tune(model, frac, start=0):
    criterion = nn.BCELoss()
    global_params = []
    persnalization_params = []
    for name, param in global_model.embeding_layer.named_parameters():
        global_params.append(param)
    for name, param in global_model.shared_layer.named_parameters():
        global_params.append(param)
    for name, param in global_model.personal_layer.named_parameters():
        persnalization_params.append(param)
    criterion = nn.BCELoss()
    lr_p=0.001
    personal_optimizer = torch.optim.NAdam(global_params, lr=lr_p, weight_decay=lr_p/10)
    personal_scheduler = optim.lr_scheduler.ReduceLROnPlateau(personal_optimizer, mode='max', factor=0.5, patience=3)

    trainer = Train(model, criterion=criterion, train_loader=train_loader, personal_optimizer=personal_optimizer, scheduler_p=personal_scheduler)
    trainer.train(epochs_list=[0,0,2],frac=frac, start=start, log=0)
def train_client(client_models, client_fractions, start=0, epoch=4):
    print("training model: ",end='')
    for i, _ in enumerate(client_models):
        print(i, end=', ')
        criterion = nn.BCELoss()
        global_params = []
        persnalization_params = []
        for name, param in global_model.embeding_layer.named_parameters():
            global_params.append(param)
        for name, param in global_model.shared_layer.named_parameters():
            global_params.append(param)
        for name, param in global_model.personal_layer.named_parameters():
            persnalization_params.append(param)
        criterion = nn.BCELoss()
        lr_p=0.001
        lr_g=0.0001
        personal_optimizer = torch.optim.NAdam(global_params, lr=lr_p, weight_decay=lr_p/10)
        global_optimizer = torch.optim.NAdam(persnalization_params, lr=lr_g, weight_decay=lr_g/10)
        personal_scheduler = optim.lr_scheduler.ReduceLROnPlateau(personal_optimizer, mode='max', factor=0.5, patience=3)
        global_scheduler = optim.lr_scheduler.ReduceLROnPlateau(global_optimizer, mode='max', factor=0.5, patience=3)

        trainer = Train(client_models[i], criterion=criterion, train_loader=train_loader, val_loader=val_loader, personal_optimizer=personal_optimizer, global_optimizer=global_optimizer, scheduler_g=global_scheduler, scheduler_p=personal_scheduler)
        trainer.train(epochs_list=[epoch,0,0],frac=client_fractions[i], val_frac=client_fractions[i], start=start, log=0)
        start += client_fractions[i]
    print()
def fed_meta_persnalization_learing(global_model, client_models, client_fractions, meta_lr=0.1):
    total_samples = sum(client_fractions)
    weights = [n_i / total_samples for n_i in client_fractions]
    global_state = {
        **global_model.embeding_layer.state_dict(),
        **global_model.shared_layer.state_dict()
    }

    avg_state = copy.deepcopy(global_state)
    for key in avg_state.keys():
        avg_state[key] = torch.zeros_like(avg_state[key])
        for client_model, w in zip(client_models, weights):
            client_state = {
                **client_model.embeding_layer.state_dict(),
                **client_model.shared_layer.state_dict()
            }
            avg_state[key] += client_state[key] * w

    new_state = {}
    for key in global_state.keys():
        new_state[key] = (1-meta_lr) * global_state[key] + meta_lr * (avg_state[key] - global_state[key])

    # 5️⃣ Load updated parameters into global model
    global_model.embeding_layer.load_state_dict({
        k: v for k, v in new_state.items() if k in global_model.embeding_layer.state_dict()
    })
    global_model.shared_layer.load_state_dict({
        k: v for k, v in new_state.items() if k in global_model.shared_layer.state_dict()
    })

    quick_fine_tune(global_model, 0.1, 0)
def update_clients(client_models, client_fractions, global_model):
    start=0.1
    for i, client in enumerate(client_models):
        client.embeding_layer.load_state_dict(global_model.embeding_layer.state_dict())
        client.shared_layer.load_state_dict(global_model.shared_layer.state_dict())
        quick_fine_tune(client, client_fractions[i], start)
        start+=client_fractions[i]


In [13]:
client_models, client_fractions = make_clients(5, total_data=0.9) 
print(client_fractions)
print('clints initial accuracy')
start = 0.1
for i, model in enumerate(client_models):
    criterion = nn.BCELoss()
    trainer = Train(model, criterion=criterion, train_loader=train_loader, val_loader=val_loader)
    avg_val_loss, val_acc = trainer.evaluate(frac=client_fractions[i], start=start, log=3)
    start += client_fractions[i]
    print(avg_val_loss, val_acc, client_fractions[i])

[0.10497028 0.4931363  0.01821473 0.00252451 0.28115418]
clints initial accuracy
starting from batch 10 ending to batch 21 total validation batch size 21
0.08814320984211835 0.9718945176960444 0.10497028132288955
starting from batch 20 ending to batch 70 total validation batch size 70
0.0830533716827631 0.9720229007633587 0.493136299220213
starting from batch 69 ending to batch 72 total validation batch size 72
0.0913119191924731 0.9688295165394402 0.018214726368543974
starting from batch 71 ending to batch 72 total validation batch size 72
0.07564617693424225 0.9770992366412213 0.0025245089054052312
starting from batch 71 ending to batch 101 total validation batch size 101
0.08442574667446635 0.9711583294478119 0.2811541841829483


In [15]:
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
for i in range(2):
    train_client(client_models, client_fractions, start=0.1)   
    fed_meta_persnalization_learing(global_model, client_models, client_fractions)
    update_clients(client_models,client_fractions, global_model)
    avg_val_loss, val_acc = trainer.evaluate(frac=0.1, start=0, log=0)
    print(f'global model loss {avg_val_loss} accuracy {val_acc}')

training model: 0, 1, 2, 3, 4, 
global model loss 0.1033929017457095 accuracy 0.9661693268563497
training model: 0, 1, 2, 3, 4, 
global model loss 0.1379835937510837 accuracy 0.9585357390700903
